In [1]:
import sys
sys.path.append("../")

In [2]:
statement_name = "../data/ozonbank_document_6732671.pdf"
incomes = 94948.0
expences = -100459

# 1. Прочитать выписку

In [3]:
from src.finman.utils.pdf import extract_text_from_pdf
from src.finman.expenses.ozon import parse_operations

In [4]:
text = extract_text_from_pdf(statement_name)
transactions_df = parse_operations(text)

In [5]:
transactions_df.head()

,id,Дата операции,Время операции,Сумма операции,Документ,Назначение платежа
0,4281248653,2024-10-30,2024-10-30 20:28:29,-3704.0,2650036929,"Оплата товаров/услуг на Платформе Ozon, заказ ..."
1,390c1475e2,2024-10-30,2024-10-30 20:27:23,4000.0,2650029559,Зачисление по переводу денежных средств по кар...
2,6d65919f51,2024-10-28,2024-10-28 21:07:20,-1568.0,2636552813,"Оплата товаров/услуг на Платформе Ozon, заказ ..."
3,36e94095e2,2024-10-28,2024-10-28 21:07:04,1000.0,2636551588,Зачисление по переводу денежных средств по кар...
4,c0ee681d97,2024-10-17,2024-10-17 13:11:54,-563.0,2542381115,"Оплата товаров/услуг на Платформе Ozon, заказ ..."


In [6]:
assert transactions_df.loc[transactions_df['Сумма операции'] > 0]['Сумма операции'].sum() == incomes
assert transactions_df.loc[transactions_df['Сумма операции'] < 0]['Сумма операции'].sum() == expences

# 2. Запроцессить гугол

In [7]:
from src.finman.utils.gsheet import GSheetWorker

In [8]:
sheet_id = "1GqwNPuOtQUXfltK6F4oHpx-U1BqBgiDZfC5YfDcJyto"
ozonbank_main_spreadsheet_name = "ozon-bank (main)"
gcreds_file = "../secrets/finman-433017-da308d823497.json"

In [9]:
gsw = GSheetWorker(gcreds_file)

In [10]:
gs_df = gsw.get_df(sheet_id, ozonbank_main_spreadsheet_name)

In [11]:
gs_df.head()

,id,Дата операции,Сумма операции,Документ,Назначение платежа


In [12]:
missing_recrods = transactions_df.loc[~transactions_df['id'].isin(gs_df['id'])]

In [13]:
missing_recrods.head()

,id,Дата операции,Сумма операции,Документ,Назначение платежа
0,4348aedc63,2024-08-16 18:21:38,-482.0,2125895187,"Оплата товаров/услуг на Платформе Ozon, заказ ..."
1,7725e5161f,2024-08-13 19:02:08,938.0,2107392458,"Возврат оплаты за товары/услуги, купленные на ..."
2,8e154900e2,2024-08-13 09:24:59,-2682.0,2103792680,"Оплата товаров/услуг на Платформе Ozon, заказ ..."
3,71275ad89c,2024-08-13 09:24:56,2690.0,1280184534,Перевод A4226062454289120000000011310501 через...
4,ead5097e9a,2024-08-12 10:29:24,-2871.0,2097210103,"Оплата товаров/услуг на Платформе Ozon, заказ ..."


In [14]:
missing_recrods['Дата операции'] = missing_recrods['Дата операции'].dt.strftime('%Y-%m-%d %H:%M:%S')

In [15]:
gsw.insert_df(missing_recrods, sheet_id, ozonbank_main_spreadsheet_name)